In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Activation, Concatenate
from keras import Sequential

from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import SGD,Adam
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

#Different Transfer learning rate/models
#https://keras.io/api/applications/
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [2]:
train_dir = "grayscale"
#test_dir = "../input/test/test/"
train_df = pd.read_csv('file_types.csv')
train_df['ImageName'] = train_df['ImageName'].astype(str)
train_df['ImageName'] = train_df['ImageName'].astype(str).str.zfill(6)
train_df['Class'] = pd.factorize(train_df['FileType'])[0] 
train_df.head()

,ImageName,FileType,Class
0,000008,ppt,0
1,000027,csv,1
2,000039,xml,2
3,000067,xml,2
4,000081,txt,3


In [3]:
width = 100
height = 100

In [4]:
import skimage.io
from PIL import Image
import cv2
import argparse

def extract_scalogram(path):
    #height =  img_height# pixels in length
    #width = img_width # pixels in width
  
    imgs = np.empty((0, width, height, 3)) 

    for filename in os.listdir(path):
        
        if filename.endswith(".png"):

            img = Image.open(os.path.join(path, filename)).convert('RGB')

            imgs = np.append(imgs, np.array(img).reshape((1, width, height, 3)), axis=0)
    
    return imgs


In [5]:
from sklearn import preprocessing

y_train = train_df['Class']

In [ ]:
X_train = extract_scalogram(train_dir)

y_train=to_categorical(y_train)

print((X_train.shape,y_train.shape))

In [ ]:
def normalize_negative_one(img):
    normalized_input = (img - np.amin(img)) / (np.amax(img) - np.amin(img))
    return 2*normalized_input - 1

In [ ]:
X_train = normalize_negative_one(X_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
    test_size=0.30, shuffle = True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,
    test_size=0.20, shuffle = True)

In [ ]:
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)
print('X_val: ', X_val.shape)
print('y_val: ', y_val.shape)

In [ ]:
batch_size= 32
epochs=100
learn_rate=.0001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

In [ ]:
train_dategen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = train_dategen.flow(X_train,  y_train, batch_size= batch_size)

validation_dategen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validation_generator = validation_dategen.flow(X_val, y_val, batch_size= batch_size)

test_generator = ImageDataGenerator(rescale = 1./255.)

In [ ]:
#Learning Rate Annealer
lrr= ReduceLROnPlateau(monitor='val_accuracy', factor=.01, patience=3, min_lr=1e-5) #change learning rATE  to get the best result

In [ ]:
#Defining the googlenet model
#use resnet - try various versions of resnet
#compare googlenet to resnet and the other 4, to see which one is better. use the latest version of them

img_width = 100
img_height = 100

base_model = InceptionV3(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
#base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3)) #
#base_model = ResNet101(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
#base_model = Xception(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height, 3))
#base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
#base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))


#use resnet - try various versions of resnet
#compare googlenet to resnet and the other 4, to see which one is better. use the latest version of them

#changing the last layer
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(14, activation='softmax'))

model.summary()


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator, 
                    epochs = epochs, 
                    steps_per_epoch = X_train.shape[0]//batch_size, 
                    validation_data = validation_generator, 
                    validation_steps = X_val.shape[0]//batch_size, callbacks=[lrr],  verbose = 1)

In [ ]:
f,ax=plt.subplots(2,1) 

#Loss
ax[0].plot(model.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(model.history.history['val_loss'],color='r',label='Validation Loss')

#Accuracy
ax[1].plot(model.history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(model.history.history['val_accuracy'],color='r',label='Validation Accuracy')